In [1]:
import os

from utils import *
from Recommenders.DataIO import DataIO
from Evaluation.Evaluator import EvaluatorHoldout

In [2]:
data = load_data()
users = load_users()
data, num_users, num_items, mapping_user_id = preprocess_data(data)
data_train, data_validation, data_test = split_data(
    data,
    num_users=num_users,
    num_items=num_items,
    validation_percentage=0.1,
    testing_percentage=0.20
)

In [3]:
evaluator_validation = EvaluatorHoldout(data_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(data_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 4171 (33.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions


In [4]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "epochs": Categorical([500]),
    "num_factors": Integer(1, 200),
    "sgd_mode": Categorical(["sgd", "adagrad", "adam"]),
    "batch_size": Categorical([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]),
    "item_reg": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "user_reg": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "learning_rate": Real(low = 1e-4, high = 1e-1, prior = 'log-uniform'),
}

In [5]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": "MAP",
                          }

In [6]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNNCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

2023-11-28 20:22:39.181822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-28 20:22:39.181841: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [data_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs,     # Additional hyperparameters for the fit function
)

In [8]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [data_validation],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs,
)

In [9]:
output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [11]:
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

recommender_class = UserKNNCFRecommender

runHyperparameterSearch_Collaborative(recommender_class,
       URM_train = data_train,
       URM_train_last_test = data_train + data_validation,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

SearchBayesianSkopt: Resuming 'UserKNNCFRecommender_cosine' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 442, 'shrink': 297, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'}
UserKNNCFRecommender: URM Detected 356 ( 2.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 194 ( 0.9%) items with no interactions.
SearchBayesianSkopt: Resuming 'UserKNNCFRecommender_jaccard' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 76, 'shrink': 672, 'similarity': 'jaccard', 'normalize': True}
UserKNNCFRecommender: URM Detected 356 ( 2.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 194 ( 0.9%) items with no interactions.
SearchBayesianSkopt: Resuming 'UserKNNCFRecommender_asymmetric' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at rand

/home/kinami/.conda/envs/RecSysFramework/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


SearchBayesianSkopt: Testing config: {'topK': 278, 'shrink': 0, 'similarity': 'jaccard', 'normalize': True}
SearchBayesianSkopt: Config 39 was already explored at index 38. Config: {'topK': 278, 'shrink': 0, 'similarity': 'jaccard', 'normalize': True} - results: PRECISION: 0.0311208, PRECISION_RECALL_MIN_DEN: 0.0996566, RECALL: 0.0975421, MAP: 0.0130877, MAP_MIN_DEN: 0.0440610, MRR: 0.1107117, NDCG: 0.0737274, F1: 0.0471867, HIT_RATE: 0.2610133, ARHR_ALL_HITS: 0.1203674, NOVELTY: 0.0044676, AVERAGE_POPULARITY: 0.4151162, DIVERSITY_MEAN_INTER_LIST: 0.8709970, DIVERSITY_HERFINDAHL: 0.9870894, COVERAGE_ITEM: 0.1498965, COVERAGE_ITEM_HIT: 0.0244802, ITEMS_IN_GT: 0.5509405, COVERAGE_USER: 0.6699636, COVERAGE_USER_HIT: 0.1748694, USERS_IN_GT: 0.6699636, DIVERSITY_GINI: 0.0151572, SHANNON_ENTROPY: 7.9746197, RATIO_DIVERSITY_HERFINDAHL: 0.9874467, RATIO_DIVERSITY_GINI: 0.0437020, RATIO_SHANNON_ENTROPY: 0.6145357, RATIO_AVERAGE_POPULARITY: 3.4322860, RATIO_NOVELTY: 0.2805106, 

EvaluatorHoldout

/home/kinami/.conda/envs/RecSysFramework/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


SearchBayesianSkopt: Testing config: {'topK': 278, 'shrink': 0, 'similarity': 'jaccard', 'normalize': True}
SearchBayesianSkopt: Config 42 was already explored at index 38. Config: {'topK': 278, 'shrink': 0, 'similarity': 'jaccard', 'normalize': True} - results: PRECISION: 0.0311208, PRECISION_RECALL_MIN_DEN: 0.0996566, RECALL: 0.0975421, MAP: 0.0130877, MAP_MIN_DEN: 0.0440610, MRR: 0.1107117, NDCG: 0.0737274, F1: 0.0471867, HIT_RATE: 0.2610133, ARHR_ALL_HITS: 0.1203674, NOVELTY: 0.0044676, AVERAGE_POPULARITY: 0.4151162, DIVERSITY_MEAN_INTER_LIST: 0.8709970, DIVERSITY_HERFINDAHL: 0.9870894, COVERAGE_ITEM: 0.1498965, COVERAGE_ITEM_HIT: 0.0244802, ITEMS_IN_GT: 0.5509405, COVERAGE_USER: 0.6699636, COVERAGE_USER_HIT: 0.1748694, USERS_IN_GT: 0.6699636, DIVERSITY_GINI: 0.0151572, SHANNON_ENTROPY: 7.9746197, RATIO_DIVERSITY_HERFINDAHL: 0.9874467, RATIO_DIVERSITY_GINI: 0.0437020, RATIO_SHANNON_ENTROPY: 0.6145357, RATIO_AVERAGE_POPULARITY: 3.4322860, RATIO_NOVELTY: 0.2805106, 

Iteration No: 43

In [24]:
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + '_tversky' + "_metadata.zip")

In [25]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 807,
 'shrink': 0,
 'similarity': 'tversky',
 'normalize': True,
 'tversky_alpha': 0.0,
 'tversky_beta': 0.6840758892980197}

In [26]:
search_metadata['result_on_test_best']

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.059567,0.108155,0.093883,0.028362,0.050123,0.188961,0.096609,0.072888,0.394985,0.229999,...,0.829957,0.327821,0.829957,0.013764,7.808504,0.985301,0.039685,0.601735,3.531594,0.280911


In [14]:
recommender_object = recommender_class(data_train + data_validation)

SLIMElasticNetRecommender: URM Detected 192 ( 1.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 127 ( 0.6%) items with no interactions.


In [15]:
recommender_object.load_model(output_folder_path, 
                              file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip" )

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIMElasticNetRecommender_best_model_last.zip'
SLIMElasticNetRecommender: Loading complete


In [16]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))
    item_ids = ratings.item_id.unique()


    recommendation_length = 10
    submission = dict()
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        recommendations = recommender.recommend(mapped_user_id, cutoff = 10)

        submission[user_id] = [mapping_to_item_id[item_id] for item_id in recommendations]
    
    for user_id in users_to_recommend:
        if user_id not in submission:
            submission[user_id] = np.random.choice(item_ids, 10)

    return submission

In [ ]:
from utils import write_submission

write_submission(prepare_submission(data, users, data_train + data_validation, recommender_object))